In [1]:
import pandas as pd
import torch.nn as nn
import torch
import copy
import networkx as nx
import numpy as np
import plotly.graph_objects as go
import os
from pickle import dump , load

In [6]:
class TaiwaneseCreditClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(23, 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 128)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(128, 32)
        self.act3 = nn.ReLU()
        self.hidden4 = nn.Linear(32, 16)
        self.act4 = nn.ReLU()
        self.output = nn.Linear(16, 1)
        self.act_output = nn.Sigmoid()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(self.act1(self.hidden1(x)))
        x = self.dropout(self.act2(self.hidden2(x)))
        x = self.dropout(self.act3(self.hidden3(x)))
        x = self.dropout(self.act4(self.hidden4(x)))
        # x = self.output(x)
        x = self.act_output(self.output(x))
        return x

    
    
def load_model():
    model = torch.load('model_training/Taiwanese_credit_model')
    model.eval()
    return model

def eval_model(model, input, scaler, columns_to_standardize):
    model.eval()
    # print(input)
    input = input.reshape(-1, 23)
    input = torch.tensor(input, dtype=torch.float32)
    input = input.numpy()
    input[:, columns_to_standardize] = scaler.transform(input[:, columns_to_standardize])
    input = torch.from_numpy(input).type(torch.float)
    with torch.no_grad():
        prob = model(input)
    return prob.tolist()[0][0]

def load_scaler(scaler_loc):
    return load(open(scaler_loc, 'rb'))

columns_to_standardize = list(range(23))


def load_data(data):
    return np.load(data)

def apply_phenotype(array, phenotype):
    # Split the phenotype string into individual operations
    operations = phenotype.split(';')
    x = copy.deepcopy(array)
    # Iterate over each operation and execute it
    for operation in operations:
        # Strip any leading/trailing whitespace from the operation
        operation = operation.strip()
        
        # Use the exec function to execute the operation on the array
        if operation:
            exec(operation)

    return x

def combine_operations(phenotype, original_input):
    combined_operations = []
    processed_input = apply_phenotype(original_input, phenotype)
    change =  processed_input - original_input
    
    # print(change)
    for i, j  in enumerate(change):
        if j == 0:
            continue
        else:
            if j > 0:
                combined_operations.append(f'x[{i}] += {j}')
            else:
                combined_operations.append(f'x[{i}] -= {-j}')
                
    return '; '.join(combined_operations)


def count_index_cooccurrences(phenotypes, num_indices):
    # Initialize the correlation matrix
    correlation_matrix = np.zeros((num_indices, num_indices), dtype=int)
    
    # Iterate over each phenotype
    for phenotype in phenotypes:
        # Split the phenotype string into individual operations
        operations = phenotype.split(';')
        
        for index, operation in enumerate(operations):
            # print(index)
            if index == len(operations) - 1:
                continue
            else:
                operation = operation.strip()
                # print(operation)
                
                if not operation:
                    continue
                
                # Parse the operation to extract the index
                current_feature = int(operation.split()[0][2:-1])
                next_feature = int(operations[index + 1].split()[0][2:-1])# Extract the index
                
                # print(current_feature, next_feature)
                
                if current_feature == next_feature:
                    continue
                else:
                    correlation_matrix[current_feature, next_feature] += 1
                    correlation_matrix[next_feature, current_feature] += 1
    

    normalized_matrix = correlation_matrix/np.linalg.norm(correlation_matrix) 
    
    return normalized_matrix, correlation_matrix


def create_graph_from_matrix(matrix):
    # Create an undirected graph
    G = nx.Graph()
    
    # Get the number of nodes
    num_nodes = matrix.shape[0]
    
    # Add nodes to the graph
    for i in range(num_nodes):
        G.add_node(i)
    
    # Add edges to the graph based on the normalized co-occurrence matrix
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if matrix[i, j] > 0:
                G.add_edge(i, j, weight=matrix[i, j])
    
    return G


def draw_graph_plotly(G, features):
    # Extract node positions
    pos = nx.spring_layout(G, seed=10)
    
    # Create edges
    edge_x = []
    edge_y = []
    edge_weights = []
    mnode_x, mnode_y, mnode_txt = [], [], []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_weights.append(edge[2]['weight'])
        mnode_x.extend([(x0 + x1)/2]) # assuming values positive/get midpoint
        mnode_y.extend([(y0 + y1)/2]) # assumes positive vals/get midpoint
        mnode_txt.extend([f"Edge ({edge[0]} - {edge[1]}): {edge[2]['weight']}"])
    
    # Create edge trace
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        mode='lines')
    
    # Create edge text for hovering
    mnode_trace = go.Scatter(x = mnode_x, y = mnode_y, mode = "markers", showlegend = False,
                         hovertemplate = " %{hovertext}<extra></extra>",
                         hovertext = mnode_txt, marker = go.scatter.Marker(opacity = 0))

    
    # Create node trace
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))
    
    # Add node connectivity as node attribute
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f'Index: {node}, feature: {features[int(node)]}<br># of connections: {len(adjacencies[1])}')
    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text
    
    # Add edge weights to edge text
    edge_trace.text = [f'Weight: {weight}' for weight in edge_weights]
    
    # Create figure
    fig = go.Figure(data=[edge_trace, node_trace, mnode_trace],
                    layout=go.Layout(
                        title='<br>Relation Graph between different features',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="Phenotype graph",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    
    # Show plot
    fig.show()

def calculate_o_1(phenotype, input, model, scaler, columns_to_standardize):
    mod_in = apply_phenotype(input, phenotype)
    o_1 = eval_model(model, mod_in, scaler, columns_to_standardize)
    return o_1

def generate_phenotype_graph(data_location, features, input_data_location, model, scaler, columns_to_standardize,  num_indices = 23):
    data = pd.read_csv(data_location)
    input = load_data(input_data_location)
    data['o_1_mod'] = data.apply(lambda row: calculate_o_1(row['Phenotype'], input, model, scaler, columns_to_standardize), axis=1)
    data = data.sort_values(by=['o_1_mod'])
    data = data[data['o_1_mod'] > 0.55]
    print(input)
    data['combined_phenotype'] = data.apply(lambda row: combine_operations(row['Phenotype'], input), axis=1)
    cooccurrence_matrix, temp = count_index_cooccurrences(list(data['Phenotype']), num_indices)
    cooccurrence_matrix = np.round(cooccurrence_matrix,decimals = 3) 
    G = create_graph_from_matrix(cooccurrence_matrix)
    draw_graph_plotly(G, features)
    return cooccurrence_matrix, G, data

def check_movement(phenotype_list, features,  features_index=23):
    # create a data dict 
    out_dict = {}
    x = [0 for i in range(23)]
    evolved_x = []
    
    for phenotype in phenotype_list:
        x_out = apply_phenotype(x, phenotype)
        evolved_x.append(x_out)
    
    # print(evolved_x)
        
    average_change = np.mean(evolved_x, axis=0)
    for i in range(features_index):
        out_dict[features[i]] =  average_change[i]
    
    return out_dict, evolved_x

def analyse_phenotype(input_data_loc, features, input_array_location, model, scaler, columns_to_standardize, features_index=23):
    co_mat, graph, data_filtered = generate_phenotype_graph(input_data_loc, features, input_array_location, model, scaler, columns_to_standardize)
    phenotype_list = data_filtered['combined_phenotype'].tolist()
    average_movement, evolved_x = check_movement(phenotype_list,features)  
    print(average_movement)
    # print(co_mat)
    return average_movement, evolved_x, co_mat, graph, data_filtered

def analyse_folder(input_folder, features_list, model, scaler, columns_to_standardize, features_index=23):
    for i in os.listdir(input_folder):
        if i=='.DS_Store':
            continue
        filename = input_folder + '/' + i + '/' + "final_gen.csv"
        input_array_location = input_folder + '/' + i + '/' + 'input_data.npy'
        print(filename)
        average_movement, evolved_x, co_mat, graph, data_filtered = analyse_phenotype(filename, features_list, input_array_location, model, scaler, columns_to_standardize)
        # print(co_mat)
        print("\n --------------------------------------- \n")
    
            
data = pd.read_csv('output/NSGAIII_multi/0/final_gen.csv')

## Lets check the relation and the movement of each feature

In [7]:
features = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11','X12', 'X13',
            'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']
input_data_loc = 'output/NSGAIII_multi/0/final_gen.csv'
input_array_location= 'output/NSGAIII_multi/0/input_data.npy'


model = load_model()
columns_to_standardize =  list(range(23))
scaler = load_scaler(scaler_loc='model_training/StandardScaler.pkl')

average_movement, evolved_x, co_mat, graph, data_filtered = analyse_phenotype(input_data_loc, features, input_array_location, model, scaler, columns_to_standardize)

[640000      0      2      2     39      0      0      0      0      0
      0 119887 123223 119211 118722 105197  93921  10000  10000  10535
  15000   5000  13627]


{'X1': 49.152542372881356, 'X2': 0.0, 'X3': 0.0, 'X4': 0.0, 'X5': -0.03389830508474576, 'X6': 2.768361581920904, 'X7': -0.011299435028248588, 'X8': 0.0, 'X9': 0.0, 'X10': 0.0, 'X11': 0.03389830508474576, 'X12': 0.0, 'X13': -20.338983050847457, 'X14': 0.0, 'X15': 0.0, 'X16': 36.15819209039548, 'X17': 305.08474576271186, 'X18': 0.0, 'X19': -15.819209039548022, 'X20': -90.3954802259887, 'X21': -56.49717514124294, 'X22': 3.389830508474576, 'X23': -40.67796610169491}


## Lets analyse all the 30 instances

In [8]:
input_data_folder = 'output/NSGAIII_multi'

analyse_folder(input_data_folder, features, model, scaler, columns_to_standardize)

output/NSGAIII_multi/20/final_gen.csv
[330000      1      1      1     34      0      0      0      0      0
      0 138009 141253 130567 128257 116988 105961   7015   7000  10007
   6500   6600   9000]


{'X1': -1651.3274336283187, 'X2': -0.004424778761061947, 'X3': 0.14601769911504425, 'X4': 0.2168141592920354, 'X5': -0.008849557522123894, 'X6': 1.924778761061947, 'X7': -0.07079646017699115, 'X8': 0.061946902654867256, 'X9': 0.0, 'X10': 0.004424778761061947, 'X11': 0.004424778761061947, 'X12': 40.70796460176991, 'X13': 61.94690265486726, 'X14': 95.13274336283186, 'X15': 95.13274336283186, 'X16': -3.982300884955752, 'X17': 236.72566371681415, 'X18': -57.52212389380531, 'X19': 36.72566371681416, 'X20': -21.238938053097346, 'X21': -91.15044247787611, 'X22': 15.044247787610619, 'X23': 62.83185840707964}

 --------------------------------------- 

output/NSGAIII_multi/18/final_gen.csv
[390000      0      1      1     38      0      0      0      0      0
      0 164418 167501 134282 128701 131529 135242   9000   7027   5000
   5000   6000   5000]


{'X1': -255.5084745762712, 'X2': 0.00847457627118644, 'X3': 0.13983050847457626, 'X4': 0.0, 'X5': 0.0, 'X6': 2.4533898305084745, 'X7': 0.00423728813559322, 'X8': -0.00847457627118644, 'X9': 0.00847457627118644, 'X10': 0.029661016949152543, 'X11': -0.00423728813559322, 'X12': 40.67796610169491, 'X13': 25.423728813559322, 'X14': 41.101694915254235, 'X15': 73.72881355932203, 'X16': 4.661016949152542, 'X17': 2.9661016949152543, 'X18': -70.33898305084746, 'X19': -5.084745762711864, 'X20': 33.898305084745765, 'X21': -5.084745762711864, 'X22': 36.016949152542374, 'X23': 53.389830508474574}

 --------------------------------------- 

output/NSGAIII_multi/27/final_gen.csv
[170000      0      1      2     28      0      0      0      0      0
      0  56766  59811  61987  63849  66276  68207   4000   3638   3500
   3500   3000   3000]


{'X1': -179.2156862745098, 'X2': 0.0, 'X3': -0.01568627450980392, 'X4': 0.0, 'X5': 0.0, 'X6': 1.811764705882353, 'X7': 0.0, 'X8': 0.03137254901960784, 'X9': 0.0196078431372549, 'X10': 0.0, 'X11': 0.03137254901960784, 'X12': 77.6470588235294, 'X13': 6.2745098039215685, 'X14': 0.0, 'X15': 2.7450980392156863, 'X16': 19.607843137254903, 'X17': 0.0, 'X18': 0.0, 'X19': 25.098039215686274, 'X20': 1.1764705882352942, 'X21': 0.0, 'X22': 0.39215686274509803, 'X23': 22.352941176470587}

 --------------------------------------- 

output/NSGAIII_multi/9/final_gen.csv
[420000      0      2      1     29      0      0      0      0      0
      0  48455  34993  35340  54763  59037  60290   2011   3000  20000
   5000   2000   3000]


{'X1': -440.74074074074076, 'X2': 0.012345679012345678, 'X3': 0.012345679012345678, 'X4': 0.0, 'X5': 0.024691358024691357, 'X6': 2.8209876543209877, 'X7': 0.030864197530864196, 'X8': 0.024691358024691357, 'X9': 0.0, 'X10': 0.04938271604938271, 'X11': 0.08024691358024691, 'X12': 79.01234567901234, 'X13': 706.1728395061729, 'X14': 314.81481481481484, 'X15': 73.45679012345678, 'X16': 55.55555555555556, 'X17': 0.0, 'X18': 37.03703703703704, 'X19': 37.03703703703704, 'X20': -1017.9012345679013, 'X21': 6.790123456790123, 'X22': 164.8148148148148, 'X23': -45.67901234567901}

 --------------------------------------- 

output/NSGAIII_multi/0/final_gen.csv
[640000      0      2      2     39      0      0      0      0      0
      0 119887 123223 119211 118722 105197  93921  10000  10000  10535
  15000   5000  13627]


...

{'X1': -124.24242424242425, 'X2': 0.005050505050505051, 'X3': 0.3181818181818182, 'X4': 0.025252525252525252, 'X5': -0.2828282828282828, 'X6': 1.8535353535353536, 'X7': -0.005050505050505051, 'X8': 0.010101010101010102, 'X9': 0.0, 'X10': 0.0, 'X11': 0.005050505050505051, 'X12': 70.20202020202021, 'X13': 14.141414141414142, 'X14': 0.0, 'X15': -3.5353535353535355, 'X16': 19.696969696969695, 'X17': 359.5959595959596, 'X18': 3.5353535353535355, 'X19': 35.35353535353536, 'X20': 0.0, 'X21': 49.494949494949495, 'X22': 3.5353535353535355, 'X23': 20.2020202020202}

 --------------------------------------- 

output/NSGAIII_multi/5/final_gen.csv
[500000      0      1      1     46      0      0      0      0      0
     -1 196606  64144  49722  67909  61613  16932  10000  10000  20025
  10000  20000  18000]


{'X1': -40.15151515151515, 'X2': 0.0, 'X3': 0.030303030303030304, 'X4': 0.015151515151515152, 'X5': 0.0, 'X6': 2.6893939393939394, 'X7': 0.007575757575757576, 'X8': 0.0, 'X9': 0.0, 'X10': 0.045454545454545456, 'X11': 0.18181818181818182, 'X12': 0.0, 'X13': 37.121212121212125, 'X14': 0.0, 'X15': 15.151515151515152, 'X16': 30.303030303030305, 'X17': 537.1212121212121, 'X18': -121.21212121212122, 'X19': 0.0, 'X20': 53.03030303030303, 'X21': -34.84848484848485, 'X22': 53.03030303030303, 'X23': -570.4545454545455}

 --------------------------------------- 

output/NSGAIII_multi/14/final_gen.csv
[210000      0      1      1     34      0      0      0      0      0
      0  74261  75602  76062  76287  76971  77737   3500   3300   3000
   3000   3000   3000]


{'X1': -238.29787234042553, 'X2': 0.0, 'X3': 0.047872340425531915, 'X4': 0.031914893617021274, 'X5': 0.031914893617021274, 'X6': 1.7127659574468086, 'X7': 0.010638297872340425, 'X8': 0.005319148936170213, 'X9': 0.06382978723404255, 'X10': 0.0, 'X11': 0.5106382978723404, 'X12': 1.5957446808510638, 'X13': 0.0, 'X14': 0.0, 'X15': 0.0, 'X16': 49.46808510638298, 'X17': 1.0638297872340425, 'X18': 0.0, 'X19': -106.38297872340425, 'X20': -37.234042553191486, 'X21': 0.0, 'X22': 17.02127659574468, 'X23': -1.0638297872340425}

 --------------------------------------- 

output/NSGAIII_multi/22/final_gen.csv
[450000      1      1      2     26      0      0      0     -1     -1
     -1  20571  37283  43482   5626  73516  11854  20000  20006   5676
  74306  11889  13347]


{'X1': 73.37662337662337, 'X2': -0.05194805194805195, 'X3': 0.0, 'X4': 0.01948051948051948, 'X5': 0.9155844155844156, 'X6': 2.7857142857142856, 'X7': 0.012987012987012988, 'X8': 0.025974025974025976, 'X9': 0.012987012987012988, 'X10': 0.05844155844155844, 'X11': 0.13636363636363635, 'X12': 12.987012987012987, 'X13': 9.090909090909092, 'X14': 72.07792207792208, 'X15': 109.0909090909091, 'X16': -1.2987012987012987, 'X17': 256.4935064935065, 'X18': 1.948051948051948, 'X19': -662.987012987013, 'X20': 0.0, 'X21': 57.142857142857146, 'X22': -16.233766233766232, 'X23': -77.92207792207792}

 --------------------------------------- 

output/NSGAIII_multi/25/final_gen.csv
[180000      0      6      2     47      0      0      0      0      0
      0 167915 163279 166994 150812 123957  55778   6028   7758   5188
   4570   1876   1701]


{'X1': 73.02631578947368, 'X2': 0.006578947368421052, 'X3': -1.743421052631579, 'X4': -0.019736842105263157, 'X5': -0.7828947368421053, 'X6': 3.0526315789473686, 'X7': 0.046052631578947366, 'X8': 0.019736842105263157, 'X9': 0.006578947368421052, 'X10': 0.006578947368421052, 'X11': 0.0, 'X12': -53.94736842105263, 'X13': -27.63157894736842, 'X14': 24.342105263157894, 'X15': 110.52631578947368, 'X16': 11.18421052631579, 'X17': 384.86842105263156, 'X18': 492.7631578947368, 'X19': -201.31578947368422, 'X20': 101.97368421052632, 'X21': 19.07894736842105, 'X22': 94.73684210526316, 'X23': 669.078947368421}

 --------------------------------------- 
